In [1]:
import gymnasium as gym
from stk_actor.wrappers import StuckStopWrapper
import torch
import torch.nn.functional as F

class PreprocessObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        """
        A Gym wrapper to preprocess mixed observation space (continuous + discrete)
        into a flat tensor.
        
        Args:
            env: The Gym environment to wrap.
        """
        super().__init__(env)
        self.observation_space = self._get_flat_observation_space(env.observation_space)
        self.mean = torch.load('buffer_mean_2', map_location='cpu')
        self.std = torch.load('buffer_std_2', map_location='cpu')

    def _get_flat_observation_space(self, observation_space):
        """
        Create a flat observation space based on the original observation space.
        
        Args:
            observation_space: Original observation space with 'continuous' and 'discrete' components.
        
        Returns:
            A flattened observation space.
        """
        continuous_dim = observation_space['continuous'].shape[0]
        discrete_dims = sum(space.n for space in observation_space['discrete'])
        flat_dim = continuous_dim + discrete_dims
        return gym.spaces.Box(low=-float('inf'), high=float('inf'), shape=(flat_dim,), dtype=float)

    def observation(self, obs):
        """
        Process the observation into a flat tensor.
        
        Args:
            obs: The raw observation from the environment.
        
        Returns:
            A preprocessed flat tensor.
        """
        continuous_obs, discrete_obs = obs['continuous'], obs['discrete']
        continuous_tensor = torch.FloatTensor(continuous_obs)
        
        discrete_tensors = [
            F.one_hot(torch.tensor(x), num_classes=num_classes.n).float()
            for x, num_classes in zip(discrete_obs, self.env.observation_space['discrete'])
        ]
        
        flat_tensor = torch.cat([continuous_tensor] + discrete_tensors)
        normed_flat_tensor = (flat_tensor - self.mean) / (self.std + 1e-8)
        return normed_flat_tensor

import gymnasium as gym
from gymnasium import Wrapper

class SkipFirstNStepsWrapper(Wrapper):
    def __init__(self, env, n):
        super().__init__(env)
        self.n = n

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)
        # Skip the first n steps
        for _ in range(self.n):
            obs, _, done, truncated, info = self.env.step(self.env.action_space.sample())
            if done or truncated:
                obs, info = self.env.reset(**kwargs)
        return obs, info

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from typing import Dict, List, Tuple, Union, Type
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gymnasium as gym
from gymnasium import spaces

def get_device(device: Union[torch.device, str] = "auto") -> torch.device:
    if device == "auto":
        device = "cuda"
    device = torch.device(device)
    if device.type == torch.device("cuda").type and not torch.cuda.is_available():
        return torch.device("cpu")
    return device

class BaseFeaturesExtractor(nn.Module):
    def __init__(self, observation_space: gym.Space, features_dim: int = 0) -> None:
        super().__init__()
        assert features_dim > 0
        self._observation_space = observation_space
        self._features_dim = features_dim
    @property
    def features_dim(self) -> int:
        return self._features_dim

def get_flattened_obs_dim(observation_space: spaces.Space) -> int:
    if isinstance(observation_space, spaces.MultiDiscrete):
        return sum(observation_space.nvec)
    else:
        return spaces.utils.flatdim(observation_space)

class FlattenExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space) -> None:
        super().__init__(observation_space, get_flattened_obs_dim(observation_space))
        self.flatten = nn.Flatten()
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.flatten(observations)
    
class MlpExtractor(nn.Module):
    def __init__(
        self,
        feature_dim: int,
        net_arch: Union[List[int], Dict[str, List[int]]],
        activation_fn: Type[nn.Module],
        device: Union[torch.device, str] = "auto",
    ) -> None:
        super().__init__()
        # device = torch.get_device(device)
        policy_net: List[nn.Module] = []
        value_net: List[nn.Module] = []
        last_layer_dim_pi = feature_dim
        last_layer_dim_vf = feature_dim

        if isinstance(net_arch, dict):
            pi_layers_dims = net_arch.get("pi", []) 
            vf_layers_dims = net_arch.get("vf", []) 
        else:
            pi_layers_dims = vf_layers_dims = net_arch
        for curr_layer_dim in pi_layers_dims:
            policy_net.append(nn.Linear(last_layer_dim_pi, curr_layer_dim))
            policy_net.append(activation_fn())
            last_layer_dim_pi = curr_layer_dim
        for curr_layer_dim in vf_layers_dims:
            value_net.append(nn.Linear(last_layer_dim_vf, curr_layer_dim))
            value_net.append(activation_fn())
            last_layer_dim_vf = curr_layer_dim

        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf
        self.policy_net = nn.Sequential(*policy_net)#.to(device)
        self.value_net = nn.Sequential(*value_net)#.to(device)

    def forward(self, features: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        :return: latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: torch.Tensor) -> torch.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: torch.Tensor) -> torch.Tensor:
        return self.value_net(features)

    
class Policy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn,):
        super().__init__()
        self.features_extractor = FlattenExtractor(observation_space)
        self.pi_features_extractor = self.features_extractor
        self.vf_features_extractor = self.features_extractor
        self.mlp_extractor = MlpExtractor(
            self.features_extractor.features_dim,
            net_arch=net_arch,
            activation_fn=activation_fn,
        )
        self.action_net = nn.Linear(net_arch[-1], sum(action_dims))
        self.value_net = nn.Linear(net_arch[-1], 1)


class UnifiedSACPolicy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn):
        super().__init__()
        
        self.shared = Policy(
            observation_space,
            action_dims,
            net_arch=net_arch,
            activation_fn=activation_fn
        )
        self.action_dims = action_dims
    
    def forward(self, x):
        x = self.shared.features_extractor(x)
        x = self.shared.mlp_extractor.policy_net(x)
        x = self.shared.action_net(x)
        return x
    
    def sample(self, x, deterministic=False):
        logits = self.forward(x)
        
        # Split logits for each action dimension
        split_logits = torch.split(logits, self.action_dims, dim=-1)
        
        actions = []
        log_probs = []
        probs = []
        
        for logit in split_logits:
            distribution = Categorical(logits=logit)
            if deterministic:
                action = torch.argmax(logit, dim=-1)
            else:
                action = distribution.sample()
            
            log_prob = distribution.log_prob(action)
            prob = F.softmax(logit, dim=-1)
            
            actions.append(action)
            log_probs.append(log_prob)
            probs.append(prob)
        
        return (
            torch.stack(actions),
            torch.stack(log_probs),
            probs
        )
    
#policy = torch.load('policy_512_512_512_512_SiLU_3_statedict', map_location='cuda')


from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from bbrl.agents.gymnasium import ParallelGymAgent, make_env
from functools import partial

tracks = [
    'abyss',
    'black_forest',
    'candela_city',
    'cocoa_temple',
    'cornfield_crossing',
    'fortmagma',
    'gran_paradiso_island',
    'hacienda',
    'lighthouse',
    'mines',
    'minigolf',
    'olivermath',
    'ravenbridge_mansion',
    'sandtrack',
    'scotland',
    'snowmountain',
    'snowtuxpeak',
    'stk_enterprise',
    'volcano_island',
    'xr591',
    'zengarden',

#   ==================   #

    'fortmagma',
    'ravenbridge_mansion',
    'snowmountain',
    'cocoa_temple',
    'sandtrack',    
    'scotland', 
    'stk_enterprise',
    'volcano_island', # 1104
    'xr591', # 864   
]

vec_env = make_vec_env(
    "supertuxkart/flattened_multidiscrete-v0",
    # seed=12,
    n_envs=8,#len(tracks), 
    wrapper_class=lambda x : (
        SkipFirstNStepsWrapper(
            StuckStopWrapper(
                PreprocessObservationWrapper(x),
                n=128,
            ), 
            n=19,
        )
    ), 
    env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), #'track':'minigolf', 
    'laps':1,
    'difficulty':2, 
    'num_kart':12, #'difficulty':0
})


# for i,venv in enumerate(vec_env.envs):
#     print(i, tracks[i%len(tracks)])
#     venv.env.default_track = tracks[i%len(tracks)]



net_arch=[1024,1024,1024]
activation_fn=torch.nn.Tanh
filename = 'policy_normed_1024_1024_1024_Tanh_statedict_2'

action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename, map_location='cpu'))


..:: Antarctica Rendering Engine 2.0 ::..


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_20261/3887382059.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load('buffer_mean_2', map_

<All keys matched successfully>

In [2]:
steps = [(
    2048,
    1_000_000,
)]
for n_steps, total_timesteps in steps:
    model = PPO(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
        device='cpu',
        learning_rate=0.0003,
        n_steps=n_steps,
        tensorboard_log="./outputs/",
        clip_range=0.2,
    )
    print('DOING', n_steps, total_timesteps)
    model.policy.load_state_dict(unified_policy.shared.state_dict())
    model.policy.load_state_dict(
        PPO.load(
            "ppti_ppo4_2048_batch128_clip01_ent0001", 
            custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), }
        ).policy.state_dict(),
    )
    model.learn(total_timesteps=total_timesteps, progress_bar=True)
    # model.save(f'ppti_ppo2_{n_steps}_batch128_clip01_ent0001')

    


Using cpu device
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
DOING 2048 1000000
Logging to ./outputs/PPO_58


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 655      |
|    ep_rew_mean     | 412      |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 347      |
|    total_timesteps | 16384    |
---------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 640       |
|    ep_rew_mean          | 442       |
| time/                   |           |
|    fps                  | 51        |
|    iterations           | 2         |
|    time_elapsed         | 633       |
|    total_timesteps      | 32768     |
| train/                  |           |
|    approx_kl            | 4.4540358 |
|    clip_fraction        | 0.524     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.29     |
|    explained_variance   | 0.833     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.599     |
|    n_updates            | 10        |
|    policy_gradient_loss | -0.0161   |
|    value_loss           | 8.85      |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 641       |
|    ep_rew_mean          | 424       |
| time/                   |           |
|    fps                  | 48        |
|    iterations           | 3         |
|    time_elapsed         | 1021      |
|    total_timesteps      | 49152     |
| train/                  |           |
|    approx_kl            | 4.6397104 |
|    clip_fraction        | 0.463     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.255    |
|    explained_variance   | 0.806     |
|    learning_rate        | 0.0003    |
|    loss                 | 1.23      |
|    n_updates            | 20        |
|    policy_gradient_loss | -0.024    |
|    value_loss           | 11.8      |
---------------------------------------


...

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 665       |
|    ep_rew_mean          | 479       |
| time/                   |           |
|    fps                  | 46        |
|    iterations           | 58        |
|    time_elapsed         | 20476     |
|    total_timesteps      | 950272    |
| train/                  |           |
|    approx_kl            | 2.3556702 |
|    clip_fraction        | 0.497     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.334    |
|    explained_variance   | 0.881     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.864     |
|    n_updates            | 570       |
|    policy_gradient_loss | -0.0207   |
|    value_loss           | 7.14      |
---------------------------------------


--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 658      |
|    ep_rew_mean          | 478      |
| time/                   |          |
|    fps                  | 46       |
|    iterations           | 59       |
|    time_elapsed         | 20904    |
|    total_timesteps      | 966656   |
| train/                  |          |
|    approx_kl            | 2.28696  |
|    clip_fraction        | 0.496    |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.342   |
|    explained_variance   | 0.864    |
|    learning_rate        | 0.0003   |
|    loss                 | 1.16     |
|    n_updates            | 580      |
|    policy_gradient_loss | -0.0221  |
|    value_loss           | 9.17     |
--------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 668       |
|    ep_rew_mean          | 476       |
| time/                   |           |
|    fps                  | 46        |
|    iterations           | 60        |
|    time_elapsed         | 21305     |
|    total_timesteps      | 983040    |
| train/                  |           |
|    approx_kl            | 2.2894773 |
|    clip_fraction        | 0.502     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.351    |
|    explained_variance   | 0.857     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.485     |
|    n_updates            | 590       |
|    policy_gradient_loss | -0.0169   |
|    value_loss           | 9.16      |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 660       |
|    ep_rew_mean          | 519       |
| time/                   |           |
|    fps                  | 45        |
|    iterations           | 61        |
|    time_elapsed         | 21760     |
|    total_timesteps      | 999424    |
| train/                  |           |
|    approx_kl            | 2.6353445 |
|    clip_fraction        | 0.493     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.322    |
|    explained_variance   | 0.876     |
|    learning_rate        | 0.0003    |
|    loss                 | 1.07      |
|    n_updates            | 600       |
|    policy_gradient_loss | -0.0244   |
|    value_loss           | 8.86      |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 651       |
|    ep_rew_mean          | 529       |
| time/                   |           |
|    fps                  | 45        |
|    iterations           | 62        |
|    time_elapsed         | 22146     |
|    total_timesteps      | 1015808   |
| train/                  |           |
|    approx_kl            | 1.8711927 |
|    clip_fraction        | 0.467     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.335    |
|    explained_variance   | 0.861     |
|    learning_rate        | 0.0003    |
|    loss                 | 1         |
|    n_updates            | 610       |
|    policy_gradient_loss | -0.0286   |
|    value_loss           | 20.6      |
---------------------------------------


In [3]:
model.save(f'ppti_ppo4_{n_steps}_batch128_clip01_ent0001')
f'ppti_ppo4_{n_steps}_batch128_clip01_ent0001'

'ppti_ppo4_2048_batch128_clip01_ent0001'